In [ ]:
### at month change, must recode in places marked fix

# uses first sheet of scb excel
### USES fhm_scb_dead.csv FHM sheet antal avlidna per dag
# fhm_dead = pd.read_csv('fhm_scb_dead.csv',sep=';',header=0,index_col=0,parse_dates=True,names=['fhm_dead'])

# historical death data from: (in scb_dead_per_month.csv)
# http://www.statistikdatabasen.scb.se/pxweb/sv/ssd/START__BE__BE0101__BE0101G/ManadFoddDod/table/tableViewLayout1/

#### !!!!! ANOMALY in SCB data: historical death data for (at least ) 2019 from statistikdatabasen 
# http://www.statistikdatabasen.scb.se/pxweb/sv/ssd/START__BE__BE0101__BE0101G/ManadFoddDod/table/tableViewLayout1/
# does not concur with death data 2019 in the preliminary data! 

### DUE to okänd dödsdag when using the daily data of the preliminary stuff - much more okänd dag than okänd månad in 
### the monthly stuff
####



### uses 2019 population for 2020 in curr_pop, multiplied with same growth factor as 2018->2019

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests

sns.set()

In [ ]:
def parse_scb_prel(YTD=True):
    
    def monthname2number(m):
        if m == 'januari' : return 1
        if m == 'februari' : return 2
        if m == 'mars' : return 3
        if m == 'april' : return 4
        if m == 'maj' : return 5
        if m == 'juni' : return 6
        if m == 'juli' : return 7
        if m == 'augusti' : return 8
        if m == 'september' : return 9
        if m == 'oktober' : return 10
        if m == 'november' : return 11
        if m == 'december' : return 12

    df = pd.read_csv('scb-dead_per_day.csv',sep=';',usecols=[1,2,3,4,5,6,7,10,11],thousands=',')
        
    df['month'] = df['Månad'].apply(monthname2number)
    df[['Månad','month']] = df[['Månad','month']].replace(np.nan,0)
    df['month'] = df['month'].astype(int)
    
    df = df.replace(np.nan,0)

    
    df[['2015','2016','2017','2018','2019','2020','2015-2019']] = df[['2015','2016','2017','2018','2019','2020','2015-2019']].astype(int)

    
    unknown_death_day = df.iloc[-1,:7]
    
    df.drop(366,inplace=True)
    
    if YTD:
        df = df.loc[df['2020'] != 0]
    
    #df = df.drop('Månad',axis=1)
    
    index = pd.date_range('2020-01-01',periods=len(df))
    df.index = index
    
    known_death_day = df.iloc[:,:7].sum()
    
    year_fraction = len(df) / 366
    
    unknown_death_day[['2015','2016','2017','2018','2019','2015-2019']] = unknown_death_day[
    ['2015','2016','2017','2018','2019','2015-2019']] * year_fraction
    
    dead_pct_per_day = df[['2015','2016','2017','2018','2019','2020','2015-2019']] / known_death_day
    
    all_deaths = df[['2015','2016','2017','2018','2019','2020','2015-2019']] + unknown_death_day * dead_pct_per_day
    
    all_deaths = pd.concat([all_deaths,df['Månad']],axis=1)
    
    return all_deaths
    

In [ ]:
# PARAM #####
last_complete_month = '2020-10-31'
curr_pop = 10327589 * 1.009521235441979 #from scb.se for 2019-12-31 * estimated growth factor


In [ ]:
def date_parser(m):
    eng_months = {'januari' : 'January',
                 'februari' : 'February',
                 'mars' : 'March',
                 'april' : 'April',
                 'maj' : 'May',
                 'juni' : 'June',
                 'juli' : 'July',
                 'augusti' : 'August',
                 'september' : 'September',
                 'oktober' : 'October',
                 'november' : 'November',
                 'december' : 'December'}
    
    eng = eng_months[m]
    
    return pd.datetime.strptime(eng,'%B').date().strftime('%B')
    
    



In [ ]:
url_0 = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101G/ManadFoddDod'

data_0 = {
  "query": [
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    },
    {
      "code": "Manad",
      "selection": {
        "filter": "item",
        "values": [
          "01",
          "02",
          "03",
          "04",
          "05",
          "06",
          "07",
          "08",
          "09",
          "10",
          "11",
          "12",
          "90"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "000000NF"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "1990",
          "1991",
          "1992",
          "1993",
          "1994",
          "1995",
          "1996",
          "1997",
          "1998",
          "1999",
          "2000",
          "2001",
          "2002",
          "2003",
          "2004",
          "2005",
          "2006",
          "2007",
          "2008",
          "2009",
          "2010",
          "2011",
          "2012",
          "2013",
          "2014",
          "2015",
          "2016",
          "2017",
          "2018",
          "2019"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r0 = requests.post(url_0,json=data_0)
r0.status_code

In [ ]:
json_0 = r0.json()
json_0['data'][0]


In [ ]:
data_list_0 = []

nr_records_0 = (len(json_0['data']))

for rec_0 in range(nr_records_0):

    data_list_0.append ((json_0['data'][rec_0]['key'][0],
                        json_0['data'][rec_0]['key'][1],json_0['data'][rec_0]['key'][2],json_0['data'][rec_0]['values'][0]))



In [ ]:
death_df = pd.DataFrame(data_list_0,columns=['gender','month','year','dead'])

death_df

In [ ]:
death_df['dead'] = death_df['dead'].astype(int)
death_df

In [ ]:
death_df = death_df.groupby(['year','month']).sum().unstack()


In [ ]:
death_df.columns = range(1,14)
death_df

In [ ]:

no_month = death_df[13]
print (no_month)
death_df = death_df.iloc[:,:-1]
death_df

In [ ]:
death_monthly_ratio = death_df.div(death_df.sum(axis=1),axis=0)
death_monthly_ratio

In [ ]:
add_deaths = death_monthly_ratio.mul(no_month,axis=0)
add_deaths

In [ ]:
death_df = death_df + add_deaths

In [ ]:
death_df.columns = ['January','February','March','April','May','June',
                    'July','August','September','October','November','December']
death_df = death_df.T

death_df

In [ ]:
death_df.iloc[:,1:].sum()

In [ ]:
(death_df.iloc[:,:].sum()).plot(kind='bar',color='r',figsize=(18,12),
                               title='SWEDEN total deaths 1990-2019')
plt.ylabel('nr of deaths')
plt.savefig('total_number_of_deaths_1990_2019.jpg',format='jpg')

In [ ]:
death_df[['2015','2016','2017','2018','2019']].sum().mean() - death_df.sum()['2019']

In [ ]:
scb_pop = pd.read_csv('scb_population.csv',encoding='ISO-8859-1')

scb_pop

In [ ]:
pd.set_option("display.precision", 2)
deaths_per_m = death_df / (scb_pop.loc[0,:] / 1e6)
deaths_per_m = deaths_per_m.T
deaths_per_m

In [ ]:
deaths_per_m.loc[:,'January' : 'September'].sum(axis=1)

deaths_per_m.sum(axis=1).plot(kind='bar',
                              figsize=(18,12),
                              color='red',
                              title='SWEDEN All Cause Deaths per million, 1990-2019')

plt.ylabel('deaths per million')

In [ ]:
monthly_deaths = []

for r,v in deaths_per_m.iterrows():
    monthly_deaths.extend(v)
    


In [ ]:
dates = pd.date_range('1990-01-01','2019-12-31',freq='M')
dates

In [ ]:
monthly_timeline = pd.DataFrame(data=monthly_deaths,index=dates,columns=['dead_per_million'])
scb_1990_2019 = monthly_timeline.copy()

In [ ]:
scb_1990_2019.loc['2019-01-01' : '2019-08-31'].sum()

In [ ]:
scb_pop.mean()['2015':'2019'].mean()

In [ ]:
scb_2020 = parse_scb_prel()
scb_2020

In [ ]:
(scb_2020.loc[:,'2015' : '2020'].sum()).astype(int)

In [ ]:
pd.date_range('2020-01-01',pd.Timestamp((pd.to_datetime(last_complete_month) + pd.DateOffset(days=31))),
                                  freq='M')

In [ ]:
last_date = scb_2020.index[-1].date()
last_date

In [ ]:
##### needs update when month changes #####

scb_2020 = scb_2020.groupby('Månad').sum()
scb_2020['min_2015_2019'] = scb_2020[['2015','2016','2017','2018','2019']].min(axis=1)
scb_2020['max_2015_2019'] = scb_2020[['2015','2016','2017','2018','2019']].max(axis=1)

scb_2020 = scb_2020[['2020','2015-2019','min_2015_2019','max_2015_2019']]
scb_2020_orig = scb_2020.copy()


####
# dividing all years 2015-2020 by 2020 population for consistency with scb_poisson
####

'''
scb_2020[['2015-2019','min_2015_2019',
          'max_2015_2019']] = scb_2020[['2015-2019','min_2015_2019',
            'max_2015_2019']] / (scb_pop.mean()['2015':'2019'].mean()/1e6)
'''

scb_2020[['2015-2019','min_2015_2019',
          'max_2015_2019']] = scb_2020[['2015-2019','min_2015_2019',
            'max_2015_2019']] / (curr_pop/1e6)

scb_2020 = pd.DataFrame(scb_2020)
scb_2020.reset_index(inplace=True)

# fix monthly
scb_2020['Månad'] = pd.Categorical(scb_2020['Månad'],['januari','februari','mars','april','maj','juni',
                                                      'juli','augusti','september','oktober','november'])
scb_2020.sort_values('Månad',inplace=True)

print (scb_2020)

# fix monthly
try:
    scb_2020.index = pd.date_range('2020-01-01',last_complete_month,freq='M')
except:
    scb_2020.index = pd.date_range('2020-01-01',pd.Timestamp((pd.to_datetime(last_complete_month) + pd.DateOffset(days=31))),
                                  freq='M')
    
#scb_2020 = scb_2020.loc[:last_complete_month]
scb_2020['2020'] = scb_2020['2020'] / (curr_pop/1e6)
scb_2020.rename(columns={'2020' : 'dead_per_million'},inplace=True)
scb_2020

In [ ]:
scb_2020['dead_per_million'].cumsum() - scb_2020['2015-2019'].cumsum()

In [ ]:
monthly_timeline

In [ ]:
monthly_timeline.loc[:'2019-12-31','dead_per_million'].plot(figsize=(18,12))
plt.plot(scb_2020.loc[:last_complete_month,'dead_per_million'],color='r')

In [ ]:
scb_2020

In [ ]:
monthly_timeline = pd.concat([monthly_timeline['dead_per_million'],scb_2020['dead_per_million']],axis=0)
monthly_timeline

In [ ]:
yearly_monthly_sums = monthly_timeline.groupby(
    [monthly_timeline.index.year,monthly_timeline.index.month]).sum()

deaths_per_m_grouped_sums_1990_2020_YTD = yearly_monthly_sums.unstack().reset_index().set_index('index')
cols = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
deaths_per_m_grouped_sums_1990_2020_YTD.columns = cols
deaths_per_m_grouped_sums_1990_2020_YTD.index.name='year'
deaths_per_m_grouped_sums_1990_2020_YTD 

In [ ]:
deaths_per_m_grouped_sums_1990_2020_YTD.to_pickle('deaths_per_m_1990-2020-YTD.pkl')

In [ ]:

deaths_per_m_grouped_sums_1990_2020_YTD.loc[:,'Jan' : 'Oct'].sum(axis=1)

In [ ]:
print (sns.axes_style())

#sns.set_style("darkgrid", {"axes.facecolor": ".9"})

sns.set_style({'grid.color' : 'lightgrey'})

In [ ]:
title = 'SWEDEN deaths per million Y2D {} stacked by month\nDataSource SCB.se'.format(last_date)

deaths_per_m_grouped_sums_1990_2020_YTD.plot(kind='bar',title=title,
                                             stacked=True,figsize=(18,12),edgecolor='none')
plt.ylabel('deaths per million')
plt.legend(loc='lower center')
plt.yticks(range(0,12000,1000))
plt.savefig('scb_1990_2020_deaths_per_M_stacked.jpg',format='jpg')



In [ ]:
two_thousands_rank = monthly_timeline.rank(ascending=False).sort_values()
two_thousands_rank = two_thousands_rank.loc['2000-01-01':]
two_thousands_rank.plot(figsize=(18,12),style=['o--'])
plt.ylabel('rank')

In [ ]:
monthly_timeline = pd.DataFrame(monthly_timeline.sort_values(ascending=False))
monthly_timeline['rank'] = range(1,len(monthly_timeline) + 1)
monthly_timeline.head(50)

In [ ]:
monthly_timeline.loc['2020-08-31']

In [ ]:
monthly_timeline.sort_index().plot(y='dead_per_million',figsize=(18,12),color=['crimson'])

In [ ]:
mean = monthly_timeline.mean()
mean

In [ ]:
monthly_timeline.loc[last_complete_month]

In [ ]:
monthly_timeline.sort_index()

In [ ]:
max_rank = monthly_timeline['rank'].max()
year_groups = monthly_timeline.groupby(pd.Grouper(freq='Y'))
year_group_2020 = (year_groups.get_group('2020-12-31 00:00:00')).copy()
year_group_2020['rel_rank'] = 1 - year_group_2020['rank'] / max_rank
year_group_2020['inv_rank'] = max_rank - year_group_2020['rank']

In [ ]:
ax = year_group_2020.set_index(year_group_2020.index.date).plot(kind='bar',y='rel_rank')

year_group_2020.set_index(year_group_2020.index.date).plot(kind='bar',y='rank')

year_group_2020.set_index(
    year_group_2020.index.date).plot(
    kind='bar',color='r',
    y='rank',
    figsize=(18,12),
    title='SWEDEN 2020 : monthly deaths per million : months ranked against all months Jan 1990 - Sept 2020 \
    \nDeadliest Month (Dec-93) ranks as\
    #1\nCurrent MAX rank (least deadly month) : {}\nHIGHER IS BETTER!'.format(len(monthly_timeline)))

plt.ylabel('Deadliest Month Rank')
plt.axhline(len(monthly_timeline),color='orange',ls='dashed',label='MAX Rank (least deadly month)')
plt.legend(loc='upper left')

plt.savefig('monthly_ranking_2020.jpg',format='jpg')

In [ ]:
year_group_2020

In [ ]:
year_groups2 = monthly_timeline.groupby(pd.Grouper(freq='M')).mean()
year_groups2

In [ ]:
top_20 = pd.DataFrame(monthly_timeline[:20])
top_20

In [ ]:
len (top_20)

In [ ]:
colors = ['b'] * 13 # must set colors after seeing which place they fall onto
colors.extend(['r','b','b','b','b','b','b'])
print (colors)
top_20.index = top_20.index.date
top_20['dead_per_million'].plot(kind='bar',
                                figsize=(18,12),
                                color=colors,
                                title='SWEDEN - Top-20 deadliest months January 1990 - {}\nDataSource: SCB.se'.format(
                                last_complete_month))
plt.ylabel('deaths per million')
plt.axhline(mean['dead_per_million'],color='orange',ls='dashed',label='average 1990-2019')
plt.legend(loc='upper right')
plt.savefig('scb_population_deaths_1990-2020.jpg',format='jpg')

In [ ]:
bottom_20 = monthly_timeline.iloc[-21:].sort_values('dead_per_million',ascending=False)
bottom_20 = bottom_20.loc[:last_complete_month]
bottom_20.index = bottom_20.index.date
bottom_20

In [ ]:
monthly_timeline.loc['2020-02-29']

In [ ]:
print ('total number of months',len(monthly_timeline))
months_rank_2020 = monthly_timeline[monthly_timeline.index.get_level_values(0).year == 2020]
months_rank_2020

In [ ]:

colors = ['b'] * 2 # must set colors after seeing which place they fall onto
colors.extend(['r','b','b','b','b','b','b','b','b','b','r','b','b','r','b','b','r','b'])


print (colors)
title = 'SWEDEN All Cause morta'
bottom_20.plot(kind='bar',y='dead_per_million',figsize=(19,12),color=colors,
              title='SWEDEN - Top-20 least deadly  months January 1990 - {}\nDataSource: SCB.se'.format(
                                last_complete_month))

plt.ylabel('deaths per million')
plt.savefig('sweden_bottom_20_deadliest_months.jpg',format='jpg')


In [ ]:

scb_1990_2019

In [ ]:
scb_1990_2020 = pd.DataFrame(pd.concat([scb_1990_2019['dead_per_million'],scb_2020['dead_per_million']],axis=0))
scb_1990_2020

In [ ]:
scb_1990_2020['year'] = scb_1990_2020.index.year
scb_1990_2020['month'] = scb_1990_2020.index.month

In [ ]:
scb_1990_2020.tail(20)

In [ ]:
month_grp = scb_1990_2020[:'2019-12-31'].groupby('month')['dead_per_million'].agg(['min','mean','max'])
print (month_grp.head(12))
scb_2020


In [ ]:
scb_2020['excess'] = scb_2020['dead_per_million'] - scb_2020['2015-2019']
scb_2020

In [ ]:
excess = scb_2020.set_index(scb_2020.index.month).join(month_grp['mean'])
excess.drop(['min_2015_2019','max_2015_2019'],axis=1,inplace=True)


In [ ]:
excess.rename(columns={'mean' : '1990-2019_mean'}, inplace=True)
excess['excess_1990_baseline'] = excess['dead_per_million'] - excess['1990-2019_mean']
excess['excess_2015_baseline'] = excess['dead_per_million'] - excess['2015-2019']
excess['cumulative_excess_vs_1990-2019'] = excess['excess_1990_baseline'].cumsum()
excess['cumulative_excess_vs_2015-2019'] = excess['excess_2015_baseline'].cumsum()

# fix monthly
#excess['date'] = pd.date_range('2020-01-01',last_complete_month,freq='M')
excess['date'] = pd.date_range('2020-01-01',pd.Timestamp((pd.to_datetime(last_complete_month) + pd.DateOffset(days=31))),
                                  freq='M')

excess[:pd.Timestamp(last_complete_month).date().month].plot(kind='bar',figsize=(18,12),
                                                             title=('Sweden Excess Deaths 2020, baselines 1990-2019 & 2015-2019'),
                                                             y=['cumulative_excess_vs_1990-2019',
                                                               'cumulative_excess_vs_2015-2019'],color=['orange','red'])
plt.ylabel('Excess Deaths per Million')
plt.xlabel('Month')
excess

In [ ]:
scb_2020['month'] = scb_2020.index.month 

plt.figure(figsize=(18,12))
plt.title('Sweden Monthly Deaths per million, adjusted for population - All Causes\nDataSource:SCB.se')
ax = plt.gca()
ax2 = plt.twinx()

ax.plot(month_grp['mean'],'o--',label='Monthly Average 1990-2019')
#ax.plot(scb_2020['month'],scb_2020['2015-2019'],'o--',color='k',label='Monthly Average 2015-2019')
#plt.fill_between(scb_2020['month'],scb_2020['min_2015_2019'],scb_2020['max_2015_2019'],color='m',alpha=0.3,label='2015-2019 range')

ax.fill_between(month_grp.index,month_grp['min'],month_grp['max'],color='c',alpha=0.3,label='1990-2019 range')

ax.plot(scb_2020['month'][:last_complete_month],scb_2020['dead_per_million'][:last_complete_month],'o--',color='crimson',label='2020')
ax.legend(loc='upper left')
ax.set_xlabel('month')
ax.set_ylabel('dead per million')

#ax2.plot(excess['cum_vs_2015'][:pd.Timestamp(last_complete_month).month],'o--',
         #color='orange',label='Excess deaths cmp 2015-2019')

ax2.plot(excess['cumulative_excess_vs_1990-2019'][:pd.Timestamp(last_complete_month).month],'o--',
         color='orange',label='Excess deaths cmp 1990-2019\nNOTE!! AXIS VALUES ARE NEGATIVE!')

ax2.set_ylabel('Cumulative Excess Deaths per Million')
#ax2.set_ylim([-400,10])
ax2.legend(loc='upper right')

xticks = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
_=plt.xticks(month_grp.index,xticks)
plt.savefig('scb_monthly_deaths_all_cmp_1990_2019.jpg',format='jpg')

In [ ]:
plt.figure(figsize=(18,12))
plt.title('Sweden Monthly Deaths per million, adjusted for population - All Causes\nDataSource:SCB.se')
ax = plt.gca()
ax2 = plt.twinx()

#ax.plot(month_grp['mean'],'o--',label='Monthly Average 1990-2019')
ax.plot(scb_2020['month'],scb_2020['2015-2019'],'o--',color='b',label='Monthly Average 2015-2019')
ax.fill_between(scb_2020['month'],scb_2020['min_2015_2019'],scb_2020['max_2015_2019'],
                color='c',alpha=0.3,label='2015-2019 range')

#ax.fill_between(month_grp.index,month_grp['min'],month_grp['max'],color='c',alpha=0.3,label='1990-2019 range')

ax.plot(scb_2020['month'][:last_complete_month],scb_2020['dead_per_million'][:last_complete_month],
        'o--',color='crimson',label='2020')
ax.legend(loc='upper left')
ax.set_xlabel('month')
ax.set_ylabel('dead per million')

#ax2.plot(excess['cum_vs_2015'][:pd.Timestamp(last_complete_month).month],'o--',
         #color='orange',label='Excess deaths cmp 2015-2019')

ax2.plot(excess['cumulative_excess_vs_2015-2019'][:],'o--',
         color='orange',label='Excess deaths cmp 2015-2019')

ax2.set_ylabel('Cumulative Excess Deaths per Million')
ax2.legend(loc='upper right')

xticks = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
_=plt.xticks(month_grp.index,xticks)
plt.savefig('scb_monthly_deaths_all_cmp_2015-2019.jpg',format='jpg')

In [ ]:
scb_2020_monthly = scb_2020.set_index('month')
print (scb_2020_monthly)

mean_1990_2019 = month_grp.loc[:pd.Timestamp(last_complete_month).month]
print (mean_1990_2019)
monthly_excess = scb_2020_monthly['dead_per_million'] - mean_1990_2019['mean']
print (monthly_excess)
print (monthly_excess.cumsum())

In [ ]:
pd.Timestamp(last_complete_month).month

In [ ]:
mask = scb_1990_2020.index.month.isin(range(1,pd.Timestamp(last_complete_month).month + 1))
first_months = scb_1990_2020[mask]['dead_per_million']
first_cum = first_months.groupby(first_months.index.year).sum()
first_cum

In [ ]:
first_cum.plot(kind='bar',figsize=(18,12),color='orange',
               title='SWEDEN cumulative deaths per million Jan 1:st - {} last,1990-2020\nDataSource:SCB.se'.format(
                   pd.Timestamp(last_complete_month).month_name()))
plt.ylabel('deaths per million')
plt.savefig('scb_deaths_jan_may_unsorted.jpg'.format('jpg'))

In [ ]:
colors = ['orange'] * 25
colors.append('crimson')
color_tail = ['orange'] * 3
colors.extend(color_tail)

print (colors)
first_cum.sort_values(ascending=False).plot(kind='bar',figsize=(18,12),
                                            title='SWEDEN cumulative deaths per million Jan 1:st - {} last,1990-2020\nDataSource:SCB.se'.format(
                   pd.Timestamp(last_complete_month).month_name()),
                                            color=colors)
plt.ylabel('deaths per million')
plt.savefig('scb_deaths_jan_july_sorted.jpg',format='jpg')

In [ ]:
first_cum.sort_values(ascending=False).plot(kind='bar',figsize=(18,12),
                                            title='SWEDEN cumulative deaths per million Jan 1:st - {} last,1990-2020\nDataSource:SCB.se'.format(
                   pd.Timestamp(last_complete_month).month_name()),
                                            color='crimson')
plt.ylabel('deaths per million')
plt.xticks(range(len(first_cum)),range(1,len(first_cum) + 1))
plt.savefig('scb_deaths_jan_july_sorted_random.jpg',format='jpg')

In [ ]:
by_month = first_months.groupby(first_months.index.month)
month_stats =[]

for month in range (1,pd.Timestamp(last_complete_month).month + 1):
    m = by_month.get_group(month)
    month_stats.append(m.agg(['min','mean','max']).values)
    
month_stats = pd.DataFrame(data=np.array(month_stats),
                           index=range(1,pd.Timestamp(last_complete_month).month + 1),
                           columns=['mini','mean','maxi'])

month_stats.index.name='month'
month_stats

In [ ]:
from matplotlib import cm

first_cumsum = first_months.groupby(first_months.index.year).cumsum()
bar = first_cumsum.groupby(first_cumsum.index.year)

plt.figure(figsize=(18,12))
plt.title('SWEDEN cumulative monthly deaths per million, 1990-2020, Jan 1st to {} last'.format(
    pd.Timestamp(last_complete_month).month_name()))

for k in bar.groups.keys():
    plt.plot(bar.get_group(k),'ro--')
    
plt.ylabel('cumulative deaths per million')

In [ ]:
first_months

In [ ]:
first_sum = first_months.groupby(first_months.index.year)

month_names = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',
              7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}

years = dict()

for k in first_sum.groups.keys():
    years[k] = first_sum.get_group(k).values
    
years = pd.DataFrame(years).T
num_months = range(1,pd.Timestamp(last_complete_month).month + 1)
years.columns= num_months

years.rename(columns=month_names,inplace=True)

years.columns.name = 'month'
years.index.name = 'year'
years_per_M = years

years_per_M

In [ ]:
years_1990_2019_mean = years.loc[1990:2019].mean().sum()
years_2015_2019_mean = years.loc[2015:2019].mean().sum()
print (years_1990_2019_mean)
print (years_2015_2019_mean)

# fix monthly
#years.drop('Oct',axis=1,inplace=True)

years

In [ ]:
years.to_csv('death_per_M_1990_2020_Jan_Jul.csv')
years.sum(axis=1)

In [ ]:


years.plot(kind='bar',stacked=True,figsize=(18,12),
          title='Sweden1990-2020 : Deaths per million : All Causes : January to {}\nDataSource:SCB.se'.format(
              pd.Timestamp(last_complete_month).month_name()))

#plt.axhline(years_1990_2019_mean,ls='dashed',color='crimson',label='1990-2019 average')
#plt.axhline(years_2015_2019_mean,ls='dashed',color='orange',label='2015-2019 average')

plt.ylabel('Nr dead per million')

plt.legend(framealpha=0.1, loc='best')
plt.yticks(range(0,10000,1000))
plt.savefig('scb_deaths_per_M_per_month_1990_2020.jpg',format='jpg')

In [ ]:
plt.figure(figsize=(18,12))
for month in years.columns:
    
    plt.plot(years.index,years[month],'o--',label=month)
    
plt.legend(loc='upper right')


In [ ]:
print (years)
deaths_per_month = years.sum(axis=0)


deaths_per_year = years.sum(axis=1) 

print (deaths_per_month)
print (deaths_per_year)

In [ ]:
deaths_per_year.plot(kind='barh',title='Sweden deaths per 10.000 Jan - {} 1990-2020'.format(
    pd.Timestamp(last_complete_month).month_name()),color='orange',
                     figsize=(18,12))

plt.xlabel('deaths per 10.000')
plt.ylabel('year')

In [ ]:
years = years_per_M
print (years[:-1])

average_2015_2019 = years.loc[2015:2019].mean()
print (average_2015_2019)
monthly_average = years[:-1].mean()
print (monthly_average)
monthly_percentiles = years[:-1].quantile(1,axis=0)
monthly_percentiles


In [ ]:
### USES fhm_scb_dead.csv

fhm_dead = pd.read_csv('fhm_scb_dead.csv',sep=';',header=0,index_col=0,parse_dates=True,names=['fhm_dead'])
fhm_dead


In [ ]:
fhm_monthly = fhm_dead.groupby(fhm_dead.index.month).sum()
fhm_monthly['fhm_dead_per_M'] = fhm_monthly['fhm_dead'] / (curr_pop / 1e6)
fhm_monthly.at[1,:] = [0,0]
fhm_monthly.at[2,:] = [0,0]
fhm_monthly.sort_index(inplace=True)
fhm_monthly

In [ ]:
scb_dead_2020_per_M = pd.DataFrame(years.loc[2020].copy())
scb_dead_2020_per_M['month_nr'] = range(1,len(scb_dead_2020_per_M) + 1)
scb_dead_2020_per_M.reset_index(inplace=True)
scb_dead_2020_per_M.set_index('month_nr',inplace=True)
scb_dead_2020_per_M

In [ ]:
all_deaths = scb_dead_2020_per_M.join(fhm_monthly)
all_deaths.rename(columns={2020 : 'scb_dead_per_M',
                          'fhm_dead_per_M' : 'covid_deaths_per_M'},inplace=True)
all_deaths['other_deaths_per_M'] = all_deaths['scb_dead_per_M'] - all_deaths['covid_deaths_per_M']
all_deaths['pct_covid_deaths'] = all_deaths['covid_deaths_per_M'] / all_deaths['scb_dead_per_M'] * 100
all_deaths['pct_other_deaths'] = all_deaths['other_deaths_per_M'] / all_deaths['scb_dead_per_M'] * 100
all_deaths

In [ ]:
(pd.Timestamp(last_complete_month) - pd.DateOffset(months=1)).month_name()

In [ ]:
all_deaths.plot(title='SWEDEN COVID-19 deaths vs OTHER deaths Jan - {} 2020\nDataSource: SCB.se, FHM'.format(
    pd.Timestamp(last_complete_month).month_name()),
                kind='bar',x='month',color=['r','orange'],
                y=['covid_deaths_per_M','other_deaths_per_M'],stacked=True,figsize=(18,12))

plt.ylabel('Total deaths per million')
plt.plot(monthly_average.index,monthly_average,'o--',label='monthly average 1990-2019',color='red')
plt.plot(average_2015_2019.index,average_2015_2019,'o--',label='monthly average 2015-2019',color='k')

plt.legend(loc='upper right')
plt.savefig('scb_fhm_covid_vs_other_per_M.jpg',format='jpg')

In [ ]:
average_2015_2019

In [ ]:
all_deaths.plot(title='SWEDEN : Percent COVID-19 deaths vs OTHER deaths Jan - {} 2020\nDataSource: SCB.se, FHM'.format(
    pd.Timestamp(last_complete_month).month_name()),
                kind='bar',x='month',color=['r','orange'],
                y=['pct_covid_deaths','pct_other_deaths'],stacked=True,figsize=(18,12))

plt.ylabel('Percent')
plt.savefig('scb_fhm_covid_vs_other_rel.jpg',format='jpg')

In [ ]:
print (years.min())
print (years.max())
print (years)


In [ ]:
years.sum(axis=1).sort_values(ascending=False)

In [ ]:
cum_mean_2015_2019 = years.sum(axis=1)
cum_mean_2015_2019 = cum_mean_2015_2019.loc[2015:2020].mean()
print (cum_mean_2015_2019)

cum_mean_1990_2019 = years.sum(axis=1)
cum_mean_1990_2019 = cum_mean_1990_2019.loc[1990:2020].mean()
print (cum_mean_1990_2019)

print (years.loc[2020].sum())

In [ ]:
scb_2020_copy = scb_2020_orig.copy()
scb_2020_copy.reset_index(inplace=True)
#scb_2020_copy['Månad'] = scb_2020_copy['Månad'].apply(date_parser)


scb_2020_copy['Månad'] = pd.Categorical(scb_2020_copy['Månad'],['januari','februari','mars','april','maj','juni',
                                                      'juli','augusti','september','oktober','november','december'])

scb_2020_copy.sort_values('Månad',inplace=True)
scb_2020_copy.drop(['min_2015_2019','max_2015_2019'],axis=1,inplace=True)

scb_2020_copy['excess_2015'] = scb_2020_copy['2020'] - scb_2020_copy['2015-2019']
scb_2020_copy['excess_2015_cum'] = scb_2020_copy['excess_2015'].cumsum()
scb_2020_copy['2020_cum'] = scb_2020_copy['2020'].cumsum()
scb_2020_copy['2020_per_M'] = scb_2020_copy['2020'] / (curr_pop / 1e6)
scb_2020_copy['2020_per_M_cum'] = scb_2020_copy['2020_per_M'].cumsum()
scb_2020_copy['excess_2015_per_M'] = scb_2020_copy['excess_2015'] / (curr_pop / 1e6)
scb_2020_copy['excess_2015_per_M_cum'] = scb_2020_copy['excess_2015_per_M'].cumsum()
scb_2020_copy

In [ ]:
death_df

In [ ]:
scb_2020_abs = scb_2020_orig.reset_index()

scb_2020_abs['Månad'] = scb_2020_abs['Månad'].apply(date_parser)
print (scb_2020_abs)

scb_2020_abs['Månad'] = pd.Categorical(scb_2020_abs['Månad'],['January','February','March',
                                                             'April','May','June','July','August','September',
                                                             'October'])
scb_2020_abs.set_index('Månad',inplace=True)

In [ ]:
scb_2020_abs.sort_index(inplace=True)
scb_2020_abs

In [ ]:
scb_2020_abs = pd.concat([death_df,scb_2020_abs['2020']],axis=1)
death_df.reset_index(inplace=True)
death_df

In [ ]:
scb_2020_abs.reset_index(inplace=True)

In [ ]:
scb_2020_abs['index'] = pd.Categorical(scb_2020_abs['index'],['January','February','March','April',
                                                             'May','June','July','August',
                                                             'September','October','November','December'])

scb_2020_abs.sort_values('index',inplace=True)
scb_2020_abs.set_index('index',inplace=True)
scb_2020_abs
                                       
                                       

In [ ]:
total_abs_ytd = scb_2020_abs[:pd.to_datetime(last_complete_month).month_name()].sum()
total_abs_ytd

In [ ]:
year_pairs_abs = total_abs_ytd.iloc[1::2].reset_index() + total_abs_ytd.iloc[2::2].reset_index()
year_pairs_abs['index'] = year_pairs_abs['index'].apply(lambda x : x[:4] + '_' + x[4:])
year_pairs_abs.columns = ['years','deaths']
year_pairs_abs

In [ ]:
pop = scb_pop.T[1:]

In [ ]:
pop = pop.iloc[::2].reset_index()
pop.columns=['year','pop']
pop

In [ ]:
year_pairs_abs['dead_per_M'] = year_pairs_abs['deaths'] / (pop['pop'] / 1e6)
year_pairs_abs.set_index('years',inplace=True)
year_pairs_abs

In [ ]:
year_pairs_abs.plot(figsize=(18,12),y='deaths',kind='bar',color='crimson',
                   title='SWEDEN sum of absolute deaths consecutive pairwise years, Jan-{} \nDataSource : SCB.se'.format(
                   pd.to_datetime(last_complete_month).month_name()))
plt.ylabel('number of deaths')

plt.savefig('pair_abs_deaths.jpg',format='jpg',dpi=400)

In [ ]:
year_pairs_abs.plot(figsize=(18,12),y='dead_per_M',kind='bar',color='crimson',
                   title='SWEDEN sum of deaths per M,consecutive pairwise years, Jan-{} \nDataSource : SCB.se'.format(
                   pd.to_datetime(last_complete_month).month_name()))
plt.ylabel('deaths per million')

plt.savefig('pair_rel_deaths.jpg',format='jpg',dpi=400)

In [ ]:
pd.to_datetime(last_complete_month).month_name()

In [ ]:
deaths_per_m_grouped_sums_1990_2020_YTD.sum(axis=1)



In [ ]:
last_date

In [ ]:
deaths_per_m_grouped_sums_1990_2020_YTD


In [ ]:
first_part_of_year = ['Aug','Sep','Oct','Nov','Dec']
second_part_of_year = ['Jan','Feb','Mar','Apr','May','Jun','Jul']


first = deaths_per_m_grouped_sums_1990_2020_YTD[first_part_of_year][1:]
second = deaths_per_m_grouped_sums_1990_2020_YTD[second_part_of_year][1:]

first = first[::2]
second = second[1::2]
print (first.sum(axis=1))
second.sum(axis=1)


In [ ]:
season_deaths_per_m = pd.concat([first.reset_index(drop=True),second.reset_index(drop=True)],axis=1)
season_deaths_per_m.sum(axis=1)

In [ ]:
title = 'SWEDEN seasonal deaths per million, August preceeding year - July subsequent year,'\
' 1991-2020\nDataSource : SCB.se'

xlabels = ['91-92','93-94','95-96','97-98','99-00','01-02','03-04','05-06','07-08',
          '09-10','11-12','13-14','15-16','17-18','19-20']

season_deaths_per_m.plot(figsize=(18,12),kind='bar',stacked=True,title=title)
plt.xticks(range(len(season_deaths_per_m.index)),xlabels)
plt.legend(loc='lower center')
plt.xlabel('Season [August-July]')
plt.ylabel('dead per million')
plt.savefig('sweden_seasonal_deaths_per_m.jpg',format='jpg',dpi=400)

In [ ]:
death_df.rename(columns={'index' : 'Månad'},inplace=True)

In [ ]:
print (scb_2020_orig)
scb_2020_orig.index = scb_2020_orig.reset_index()['Månad'].apply(date_parser)

In [ ]:
scb_2020_orig

In [ ]:
abs_deaths = death_df.set_index('Månad').join(scb_2020_orig['2020'])

In [ ]:
abs_deaths = abs_deaths.T
abs_deaths

In [ ]:
first_season_half = abs_deaths.loc[:,'September' : 'December'][1::2]
second_season_half = abs_deaths.loc[:,: 'August'][2::2]

print (first_season_half)
second_season_half

In [ ]:
season_deaths_abs= pd.concat([first_season_half.reset_index(drop=True),second_season_half.reset_index(drop=True)],axis=1)
season_deaths_abs.sum(axis=1)

In [ ]:
scb_pop['2020'] = scb_pop['2019']

In [ ]:
ave_pop = ((scb_pop.T + scb_pop.T.shift()) / 2)[::2].reset_index(drop=True).dropna().reset_index(drop=True)
ave_pop

In [ ]:
season_deaths_abs = season_deaths_abs[['January','February','March','April','May','June',
                                      'July','August','September','October','November','December']]

season_deaths_abs.index = ['91-92','93-94','95-96','97-98','99-00','01-02','03-04',
                          '05-06','07-08','09-10','11-12','13-14','15-16',
                          '17-18','19-20']

season_deaths_abs

In [ ]:
season_tot_deaths_abs = season_deaths_abs.sum(axis=1)
season_tot_deaths_per_m = season_tot_deaths_abs.reset_index(drop=True) / (ave_pop[0] / 1e6)

season_tot_deaths_per_m.index = ['91-92','93-94','95-96','97-98','99-00','01-02','03-04',
                          '05-06','07-08','09-10','11-12','13-14','15-16',
                          '17-18','19-20']
season_tot_deaths_per_m

In [ ]:
season_tot_deaths_per_m.plot(figsize=(18,12),kind='bar')

In [ ]:
second_season_half


In [ ]:
abs_deaths

In [ ]:
first_season_half_even = abs_deaths.loc[:,'November' : 'December'][2::2]
first_season_half_even = first_season_half_even[:-1]
second_season_half_even = abs_deaths.loc[:,: 'October'][3::2]

print (first_season_half_even)
second_season_half_even

In [ ]:
season_deaths_abs_even = pd.concat([first_season_half_even.reset_index(drop=True),
                                    second_season_half_even.reset_index(drop=True)],axis=1)

season_deaths_abs_even


In [ ]:
season_deaths_abs_even = season_deaths_abs_even[['January','February','March','April','May','June',
                                      'July','August','September','October','November','December']]

season_deaths_abs_even.index = ['92-93','94-95','96-97','98-99','00-01','02-03','04-05',
                          '06-07','08-09','10-11','12-13','14-15','16-17',
                          '18-19']

season_deaths_abs_even

In [ ]:
season_even_tot_deaths_abs = season_deaths_abs_even.sum(axis=1)
season_even_tot_deaths_abs

In [ ]:
scb_pop.T

In [ ]:
ava_pop_even = ((scb_pop.T.loc['1992':] + scb_pop.T.loc['1992':].shift(-1)) / 2)[::2].dropna().astype(int).reset_index(drop=True)
ava_pop_even

In [ ]:
season_even_tot_deaths_per_m = season_even_tot_deaths_abs.reset_index(drop=True) / (ava_pop_even[0] / 1e6)
season_even_tot_deaths_per_m.dropna(inplace=True)

season_even_tot_deaths_per_m

In [ ]:
season_even_tot_deaths_per_m.index = ['92-93','94-95','96-97','98-99','00-01','02-03','04-05',
                          '06-07','08-09','10-11','12-13','14-15','16-17',
                          '18-19']

season_even_tot_deaths_per_m

In [ ]:
all_season_deaths_per_m = pd.concat([season_tot_deaths_per_m,season_even_tot_deaths_per_m])
all_season_deaths_per_m

In [ ]:
new_index = ['91-92','92-93','93-94','94-95','95-96','96-97','97-98','98-99',
            '99-00','00-01','01-02','02-03','03-04','04-05','05-06','06-07',
            '07-08','08-09','09-10','10-11','11-12','12-13','13-14','14-15',
            '15-16','16-17','17-18','18-19','19-20']

all_season_deaths_per_m = all_season_deaths_per_m.reindex(new_index)
all_season_deaths_per_m

In [ ]:
all_season_deaths_per_m.plot(figsize=(18,12),kind='bar',color='crimson',
                            title='SWEDEN Seasonal All Cause Deaths 1991/1992 - 2019/2020, Oct-Sep')

plt.xlabel('Season [Oct -> Sep]')
plt.ylabel('deaths per million')

plt.savefig('sweden_seasonal_deaths_91_2020_sep_aug.jpg',format='jpg',dpi=400)
                             

In [ ]:
scb = parse_scb_prel(YTD=False)
scb = scb.loc[:,'2015':'2015-2019'].astype(int)
scb['2015-2018'] = scb.loc[:,'2015':'2018'].mean(axis=1)
scb

In [ ]:
np.argwhere(scb.index=='2020-09-25')

In [ ]:
scb.sum()

In [ ]:
cum_means = scb.cumsum().div(np.arange(1,len(scb) + 1),axis=0)

In [ ]:
cum_means.plot(y=['2015','2020'],title='Proportional daily running mean YTD')
plt.ylabel('average daily deaths at day n')

In [ ]:
cum_means

In [ ]:
(cum_means.loc['2020-09-04','2020'] - cum_means.loc['2020-09-04','2015-2019']) 

In [ ]:
scb_pop

In [ ]:
scb.drop(['2015-2018','2015-2019'],axis=1,inplace=True)
scb

In [ ]:
pop_15_20 = scb_pop.loc[:,'2014' : '2019']
pop_15_20.columns=['2015','2016','2017','2018','2019','2020']
pop_15_20

In [ ]:
scb_per_M = scb / (pop_15_20.values / 1e6)
scb_per_M = scb_per_M[:-4]
scb_per_M

In [ ]:
cum_means_per_M = scb_per_M.cumsum().div(np.arange(1,len(scb_per_M) + 1),axis=0)
cum_means_per_M_avg = cum_means_per_M.loc[:,'2015' : '2018'].mean(axis=1)
cum_means_per_M

In [ ]:
cum_means_per_M.loc[:'2020-09-18']

In [ ]:
last_d = cum_means_per_M.index[-1].date()
last_d

In [ ]:
last_date = last_date - pd.DateOffset(weeks=1)

In [ ]:
colors = ['r','g','b','yellow','orange','k']
ax = cum_means_per_M.loc[:last_date].plot(
    title='SWEDEN Moving daily average deaths per million YTD {}, 2015-2020'.format(last_date),
                    figsize=(18,12),color=colors)

cum_means_per_M_avg.plot(ax=ax,style='k--',label='2015-2018 mean',lw=3)

plt.ylabel('average YTD daily deaths per million on day n of year')
plt.axvline(last_date,color='orange',ls='dashed',label='2020 last data: {}'.format(last_date))
plt.legend(loc='upper right')
plt.savefig('running_daily_average_deaths.jpg',format='jpg')

In [ ]:
total_abs_ytd

In [ ]:
pair_wise_years_abs = total_abs_ytd + total_abs_ytd.shift()
pair_wise_years_abs.dropna(inplace=True)
pair_wise_years_abs.index = ['90/91','91/92','92/93','93/94','94/95','95/96','96/97','97/98',
                            '98/99','99/00','00/01','01/02','02/03','03/04','04/05','05/06',
                            '06/07','07/08','08/09','09/10','10/11','11/12','12/13','13/14',
                            '14/15','15/16','16/17','17/18','18/19','19/20']

pair_wise_years_abs / 2


In [ ]:
scb_pop

In [ ]:
pair_wise_pop = (scb_pop.T + scb_pop.T.shift()) / 2
pair_wise_pop.dropna(inplace=True)
pair_wise_pop.index = ['90/91','91/92','92/93','93/94','94/95','95/96','96/97','97/98',
                            '98/99','99/00','00/01','01/02','02/03','03/04','04/05','05/06',
                            '06/07','07/08','08/09','09/10','10/11','11/12','12/13','13/14',
                            '14/15','15/16','16/17','17/18','18/19','19/20']

pair_wise_pop = pair_wise_pop[0]
pair_wise_pop

In [ ]:
pair_wise_deaths_per_m = (pair_wise_years_abs / 2) / (pair_wise_pop / 1e6)
pair_wise_deaths_per_m

In [ ]:
pd.to_datetime(last_complete_month).month_name()

In [ ]:
title='SWEDEN Mortality - "Regression to the Mean" :'\
'\naverage deaths per million January - {}\n'\
'pairwise consecutive years, 1990-2020\nDataSource : SCB.se'.format(
    pd.to_datetime(last_complete_month).month_name())

pair_wise_deaths_per_m.plot(kind='bar',figsize=(18,12),title=title,color='crimson')
plt.ylabel('average deaths per million for two consecutive years')
plt.savefig('sweden_regression_to_mean.jpg',format='jpg')

In [ ]:
print (years_per_M.min())
print (years_per_M.apply(lambda x : years_per_M.index[x.argmin()]))

In [ ]:
years_per_M

In [ ]:
odd_years = years_per_M.iloc[1::2]
even_years = years_per_M.iloc[2::2]
print (odd_years)
even_years

In [ ]:
pairwise_years = (odd_years.reset_index(drop=True) + even_years.reset_index(drop=True)) / 2
pairwise_years.index=['91/92','93/94','95/96','97/98','99/00',
                     '01/02','03/04','05/06','07/08','09/10',
                     '11/12','13/14','15/16','17/18','19/20']
pairwise_years

In [ ]:
pairwise_years.sum(axis=1)

In [ ]:
month = pd.to_datetime(last_complete_month).month_name()
day = pd.to_datetime(last_complete_month).day

In [ ]:
title = 'SWEDEN avg deaths per M, pairwise years, January 1:st - {} {}:th\nDataSource : scb.se'.format(month,day)
pairwise_years.sum(axis=1).plot(kind='bar',figsize=(18,12),color='crimson',title=title)
plt.ylabel('deaths per million')
plt.xlabel('paired years')

plt.savefig('swe_pairwise_years_deaths_per_m_ytd.jpg',format='jpg')

In [ ]:
abs_deaths_ytd = abs_deaths.loc[:,:'October'].sum(axis=1)
abs_deaths_ytd

In [ ]:
odd_abs = abs_deaths_ytd.iloc[1::2]
even_abs = abs_deaths_ytd.iloc[2::2]
print (odd_abs)
print (even_abs)

In [ ]:
pairwise_abs = (odd_abs.reset_index(drop=True) + even_abs.reset_index(drop=True)) / 2
pairwise_abs.index=['91/92','93/94','95/96','97/98','99/00',
                     '01/02','03/04','05/06','07/08','09/10',
                     '11/12','13/14','15/16','17/18','19/20']

pairwise_abs

In [ ]:
pairwise_abs.plot(kind='bar',figsize=(18,12))